In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [8]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('true.csv')

In [9]:
data_fake.head()

,id,title,author,text,state,date_published,source,category,sentiment_score,word_count,...,num_shares,num_comments,political_bias,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label
0,1,Breaking News 1,Jane Smith,This is the content of article 1. It contains ...,Tennessee,30-11-2021,The Onion,Entertainment,-0.22,1302,...,47305,450,Center,FALSE,1,76,6,0.84,53.35,Fake
1,2,Breaking News 2,Emily Davis,This is the content of article 2. It contains ...,Wisconsin,02-09-2021,The Guardian,Technology,0.92,322,...,39804,530,Left,Mixed,1,1,5,0.85,28.28,Fake
2,3,Breaking News 3,John Doe,This is the content of article 3. It contains ...,Missouri,13-04-2021,New York Times,Sports,0.25,228,...,45860,763,Center,Mixed,0,57,1,0.72,0.38,Fake
3,4,Breaking News 4,Alex Johnson,This is the content of article 4. It contains ...,North Carolina,08-03-2020,CNN,Sports,0.94,155,...,34222,945,Center,TRUE,1,18,10,0.92,32.20,Fake
4,5,Breaking News 5,Emily Davis,This is the content of article 5. It contains ...,California,23-03-2022,Daily Mail,Technology,-0.01,962,...,35934,433,Right,Mixed,0,95,6,0.66,77.70,Real


In [10]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [51]:
data_fake["class"] = 0
data_true['class'] = 1

In [52]:
data_fake.shape, data_true.shape

((4000, 25), (21417, 5))

In [53]:
data_fake_manual_testing = data_fake.tail(10)
data_fake = data_fake.iloc[:-10]
    
data_true_manual_testing = data_true.tail(10)
data_true = data_true.iloc[:-10]

In [54]:
print("Fake data shape after removal:", data_fake.shape)
print("True data shape after removal:", data_true.shape)

Fake data shape after removal: (3990, 25)
True data shape after removal: (21407, 5)


In [55]:
data_fake_manual_testing = data_fake.tail(10).copy()
data_fake_manual_testing['class'] = 0
data_true_manual_testing = data_true.tail(10).copy()
data_true_manual_testing['class'] = 1

In [56]:
data_fake_manual_testing.head(10)

,id,title,author,text,state,date_published,source,category,sentiment_score,word_count,...,num_comments,political_bias,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label,class
3980,3981,Breaking News 3981,Jane Smith,This is the content of article 3981. It contai...,Indiana,11-11-2023,BBC,Technology,0.32,1236,...,321,Center,Mixed,1,99,10,0.85,50.11,Fake,0
3981,3982,Breaking News 3982,Emily Davis,This is the content of article 3982. It contai...,New Jersey,29-10-2023,Reuters,Business,0.34,1493,...,502,Center,TRUE,0,5,5,0.02,83.75,Fake,0
3982,3983,Breaking News 3983,Alex Johnson,This is the content of article 3983. It contai...,Arizona,14-01-2021,New York Times,Business,0.94,976,...,800,Left,Mixed,1,39,5,0.54,22.18,Real,0
3983,3984,Breaking News 3984,Alex Johnson,This is the content of article 3984. It contai...,North Carolina,08-06-2021,CNN,Business,-0.59,1496,...,438,Right,TRUE,1,17,3,0.66,96.18,Fake,0
3984,3985,Breaking News 3985,John Doe,This is the content of article 3985. It contai...,Texas,12-08-2021,InfoWars,Sports,0.91,392,...,952,Left,FALSE,1,46,9,0.85,16.78,Fake,0
3985,3986,Breaking News 3986,Emily Davis,This is the content of article 3986. It contai...,Virginia,04-04-2020,The Onion,Entertainment,0.98,1059,...,462,Left,FALSE,1,0,1,0.59,2.11,Fake,0
3986,3987,Breaking News 3987,Chris Lee,This is the content of article 3987. It contai...,Pennsylvania,18-04-2022,Daily Mail,Politics,-0.96,1289,...,883,Left,TRUE,1,73,7,0.18,14.53,Real,0
3987,3988,Breaking News 3988,Jane Smith,This is the content of article 3988. It contai...,Florida,02-08-2022,Washington Post,Health,0.08,1289,...,904,Left,TRUE,0,22,9,0.58,35.04,Real,0
3988,3989,Breaking News 3989,Alex Johnson,This is the content of article 3989. It contai...,Texas,17-09-2022,Snopes,Sports,0.24,1484,...,325,Right,FALSE,1,84,7,0.26,0.68,Real,0
3989,3990,Breaking News 3990,John Doe,This is the content of article 3990. It contai...,Washington,18-02-2024,The Onion,Entertainment,-0.26,1209,...,689,Right,Mixed,0,35,10,0.13,57.86,Real,0


In [57]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [58]:
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)

,id,title,author,text,state,date_published,source,category,sentiment_score,word_count,...,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label,class,subject,date
0,1.0,Breaking News 1,Jane Smith,This is the content of article 1. It contains ...,Tennessee,30-11-2021,The Onion,Entertainment,-0.22,1302.0,...,FALSE,1.0,76.0,6.0,0.84,53.35,Fake,0,NaN,NaN
1,2.0,Breaking News 2,Emily Davis,This is the content of article 2. It contains ...,Wisconsin,02-09-2021,The Guardian,Technology,0.92,322.0,...,Mixed,1.0,1.0,5.0,0.85,28.28,Fake,0,NaN,NaN
2,3.0,Breaking News 3,John Doe,This is the content of article 3. It contains ...,Missouri,13-04-2021,New York Times,Sports,0.25,228.0,...,Mixed,0.0,57.0,1.0,0.72,0.38,Fake,0,NaN,NaN
3,4.0,Breaking News 4,Alex Johnson,This is the content of article 4. It contains ...,North Carolina,08-03-2020,CNN,Sports,0.94,155.0,...,TRUE,1.0,18.0,10.0,0.92,32.20,Fake,0,NaN,NaN
4,5.0,Breaking News 5,Emily Davis,This is the content of article 5. It contains ...,California,23-03-2022,Daily Mail,Technology,-0.01,962.0,...,Mixed,0.0,95.0,6.0,0.66,77.70,Real,0,NaN,NaN
5,6.0,Breaking News 6,John Doe,This is the content of article 6. It contains ...,North Carolina,23-03-2022,New York Times,Sports,0.83,920.0,...,FALSE,0.0,8.0,1.0,0.01,72.10,Fake,0,NaN,NaN
6,7.0,Breaking News 7,Emily Davis,This is the content of article 7. It contains ...,Maryland,06-10-2022,Fox News,Business,0.81,651.0,...,Mixed,0.0,1.0,10.0,0.47,97.59,Fake,0,NaN,NaN
7,8.0,Breaking News 8,John Doe,This is the content of article 8. It contains ...,Maryland,25-09-2022,The Onion,Politics,-0.96,717.0,...,TRUE,1.0,79.0,5.0,0.58,75.33,Real,0,NaN,NaN
8,9.0,Breaking News 9,John Doe,This is the content of article 9. It contains ...,Tennessee,30-08-2021,HuffPost,Politics,-0.64,1093.0,...,TRUE,1.0,96.0,7.0,0.08,39.37,Fake,0,NaN,NaN
9,10.0,Breaking News 10,Jane Smith,This is the content of article 10. It contains...,Maryland,18-03-2022,New York Times,Business,-0.50,1421.0,...,FALSE,1.0,88.0,3.0,0.68,99.12,Real,0,NaN,NaN


In [59]:
data_fake['class'] = 0
data_true['class'] = 1
data_merge = pd.concat([data_fake, data_true], axis=0)
print(data_merge.columns)

Index(['id', 'title', 'author', 'text', 'state', 'date_published', 'source',
       'category', 'sentiment_score', 'word_count', 'char_count', 'has_images',
       'has_videos', 'readability_score', 'num_shares', 'num_comments',
       'political_bias', 'fact_check_rating', 'is_satirical', 'trust_score',
       'source_reputation', 'clickbait_score', 'plagiarism_score', 'label',
       'class', 'subject', 'date'],
      dtype='object')


In [60]:
data = data_merge.drop(['title','subject', 'date'],axis=1)

In [61]:
data.isnull().sum()

id                   21407
author               21407
text                     0
state                21407
date_published       21407
source               21407
category             21407
sentiment_score      21407
word_count           21407
char_count           21407
has_images           21407
has_videos           21407
readability_score    21407
num_shares           21407
num_comments         21407
political_bias       21407
fact_check_rating    21407
is_satirical         21407
trust_score          21407
source_reputation    21407
clickbait_score      21407
plagiarism_score     21407
label                21407
class                    0
dtype: int64

In [62]:
data = data.sample(frac = 1)

In [63]:
data.head()

,id,author,text,state,date_published,source,category,sentiment_score,word_count,char_count,...,num_comments,political_bias,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label,class
19758,NaN,NaN,JOHANNESBURG (Reuters) - A sudden decision for...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9872,NaN,NaN,WASHINGTON (Reuters) - U.S. House Speaker Paul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
11951,NaN,NaN,TOKYO (Reuters) - Japan formally decided on Tu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16719,NaN,NaN,BEIRUT (Reuters) - Syrian President Bashar al-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19643,NaN,NaN,MEXICO CITY (Reuters) - At least four people w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [64]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [65]:
data.columns

Index(['id', 'author', 'text', 'state', 'date_published', 'source', 'category',
       'sentiment_score', 'word_count', 'char_count', 'has_images',
       'has_videos', 'readability_score', 'num_shares', 'num_comments',
       'political_bias', 'fact_check_rating', 'is_satirical', 'trust_score',
       'source_reputation', 'clickbait_score', 'plagiarism_score', 'label',
       'class'],
      dtype='object')

In [66]:
data.head(10)

,id,author,text,state,date_published,source,category,sentiment_score,word_count,char_count,...,num_comments,political_bias,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label,class
0,NaN,NaN,JOHANNESBURG (Reuters) - A sudden decision for...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,WASHINGTON (Reuters) - U.S. House Speaker Paul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,TOKYO (Reuters) - Japan formally decided on Tu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NaN,NaN,BEIRUT (Reuters) - Syrian President Bashar al-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,NaN,NaN,MEXICO CITY (Reuters) - At least four people w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,97.0,Chris Lee,This is the content of article 97. It contains...,Arizona,01-09-2022,BBC,Politics,0.1,104.0,4978.0,...,347.0,Right,FALSE,1.0,60.0,10.0,0.14,15.05,Fake,0
6,NaN,NaN,WASHINGTON (Reuters) - The collapse of Islamic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,NaN,NaN,WASHINGTON (Reuters) - Senate Judiciary Commit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,NaN,NaN,WASHINGTON (Reuters) - U.S. President Donald T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,NaN,NaN,WASHINGTON (Reuters) - The U.S. Senate on Thur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [67]:
def wordopt(text):
    text = re.sub(r'\[.*?\]', '', text)                              # remove content in brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)               # remove URLs
    text = re.sub(r'<.*?>+', '', text)                               # remove HTML tags
    text = re.sub(r'[%s]' % re.escape(r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)  # remove punctuation
    text = re.sub(r'\n', '', text)                                   # remove newlines
    text = re.sub(r'\w*\d\w*', '', text)                             # remove words with digits
    return text

In [28]:
data['text'] = data['text'].apply(wordopt)

In [29]:
x = data['text']
y = data['class']

In [30]:
x_train ,x_test ,y_train ,y_test = train_test_split(x,y, test_size= 0.25)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [32]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [33]:
pred_lr = LR.predict(xv_test)

In [34]:
LR.score(xv_test, y_test)

1.0

In [35]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1018
           1       1.00      1.00      1.00      5337

    accuracy                           1.00      6355
   macro avg       1.00      1.00      1.00      6355
weighted avg       1.00      1.00      1.00      6355



In [36]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [37]:
pred_DT = DT.predict(xv_test)

In [38]:
DT.score(xv_test, y_test)

1.0

In [39]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1018
           1       1.00      1.00      1.00      5337

    accuracy                           1.00      6355
   macro avg       1.00      1.00      1.00      6355
weighted avg       1.00      1.00      1.00      6355



In [40]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [41]:
predict_gb = GB.predict(xv_test)

In [42]:
GB.score(xv_test, y_test)

1.0

In [43]:
print(classification_report(y_test, predict_gb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1018
           1       1.00      1.00      1.00      5337

    accuracy                           1.00      6355
   macro avg       1.00      1.00      1.00      6355
weighted avg       1.00      1.00      1.00      6355



In [44]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [45]:
predict_rf = RF.predict(xv_test)

In [46]:
RF.score(xv_test, y_test)

1.0

In [47]:
print(classification_report(y_test, predict_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1018
           1       1.00      1.00      1.00      5337

    accuracy                           1.00      6355
   macro avg       1.00      1.00      1.00      6355
weighted avg       1.00      1.00      1.00      6355



In [48]:
def output_label(n):
    if n == 0:
        return " Fake News"
    elif n == 1:
        return " Not a Fake News"
    else:
        return " Unknown"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GB.predict(new_xv_test)
    pred_RFC = RF.predict(new_xv_test)

    print("\n\nLR Prediction: {}\nDT Prediction: {}\nGBC Prediction: {}\nRFC Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GBC[0]),
        output_label(pred_RFC[0])
    ))



In [68]:
import joblib

joblib.dump(LR, "logistic_model.pkl")
joblib.dump(DT, "decision_tree_model.pkl")
joblib.dump(GB, "gradient_boost_model.pkl")
joblib.dump(RF, "random_forest_model.pkl")
joblib.dump(vectorization, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']